In [1]:
import requests
import pandas as pd
import os
from bs4 import BeautifulSoup
from string import punctuation
from nltk.tokenize import word_tokenize

In [2]:
df=pd.read_excel("Input.xlsx")
directory = "StopWords/"
b=[]
for file in os.listdir(directory):
    a=open("{0}{1}".format(directory,file),encoding="ISO-8859-1")
    b=b+list(word_tokenize(a.read()))
c=list(set(b))   

In [3]:
def clean(sentiment,stop_words,punctuation): 
    punctuation= punctuation + "\n"
    tokens = word_tokenize(sentiment)
    data=[]
    for word in tokens:    
        if word.lower() not in stop_words:
            if word.lower() not in punctuation:
                data.append(word)
    return data

In [4]:

def analysis(sentiment,c,punctuation):
    p=0
    n=0
    data=clean(sentiment,c,punctuation)
    a=open("MasterDictionary/positive-words.txt",encoding="ISO-8859-1")
    positive_words=list(word_tokenize(a.read()))
    b=open("MasterDictionary/negative-words.txt",encoding="ISO-8859-1")
    negative_words=list(word_tokenize(b.read()))
    for word in data:
        if word in positive_words:
            p=p+1
        elif word in negative_words:
            n=n+1
    return p,n

In [5]:
p=[]
n=[]
for i in range(df.shape[0]):
    url=df["URL"][i]
    req = requests.get(url)
    soup=BeautifulSoup(req.text,"html.parser")
    try:
        title=soup.find_all("header")[0].find_all("h1")[0].text
        body=title+"\n"
        for para in soup.find_all(class_="td-post-content tagdiv-type")[0].find_all("p"):
            body=body+"\n"+para.text
        text_file = open("MiningData/{}.txt".format(df["URL_ID"][i]), "w")
        text_file.write(body)
        text_file.close()
        pos,neg=analysis(body.lower(),c,punctuation)
        p.append(pos)
        n.append(neg)
            
    except Exception as e:
        try:
            title=soup.find_all(class_="tdb-title-text")[0].text
            body=title+"\n"
            for para in soup.find_all(class_="tdb-block-inner td-fix-index")[0].find_all("p"):
                body=body+"\n"+para.text
            text_file = open("MiningData/{}.txt".format(df["URL_ID"][i]), "w")
            text_file.write(body)
            text_file.close()
            pos,neg=analysis(body.lower(),c,punctuation)
            p.append(pos)
            n.append(neg)
        except:
            title=soup.find_all(class_="tdm-title tdm-title-md")[0].text
            body=title+"\n"
            for para in soup.find_all(class_="tdm_block td_block_wrap tdm_block_inline_text tdi_118 td-pb-border-top td_block_template_1")[0].find_all("p"):
                body=body+"\n"+para.text
            text_file = open("MiningData/{}.txt".format(df["URL_ID"][i]), "w")
            text_file.write(body)
            text_file.close()
            p.append(0)
            n.append(0)

In [6]:
df["pos"]=p
df["neg"]=n
df.head()

,URL_ID,URL,pos,neg
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,80,24
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,38,13
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,0,0
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,34,27
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,34,27


In [7]:
df["Polarity Score"]=(df["pos"]-df["neg"])/((df["pos"]+df["neg"])+0.000001)
df.head()

,URL_ID,URL,pos,neg,Polarity Score
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,80,24,0.538462
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,38,13,0.490196
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,0,0,0.000000
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,34,27,0.114754
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,34,27,0.114754
